# Optimizing protein stability using random mutations

## Stability optimization is a registered problem


`poli`'s registry has a `get_problems()` method which returns the objective functions already compiled. It works by loading up a `config.rc` file sitting inside `src/poli/core`.

In [1]:
from poli.core.registry import get_problems
get_problems(only_available=False)

['aloha',
 'dockstring',
 'drd3_docking',
 'foldx_rfp_lambo',
 'foldx_sasa',
 'foldx_stability',
 'foldx_stability_and_sasa',
 'gfp_cbas',
 'gfp_select',
 'penalized_logp_lambo',
 'rasp',
 'rdkit_logp',
 'rdkit_qed',
 'rfp_foldx_stability_and_sasa',
 'sa_tdc',
 'super_mario_bros',
 'toy_continuous_problem',
 'white_noise']

As you can see, `foldx_stability` is already available in the repository.

Let's stick with it as a problem name:

In [2]:
problem_name = "foldx_stability"

## Optimizing `mRouge`

In this example, we will focus on optimizing [`mRouge`, also known as `3NED`](https://www.rcsb.org/structure/3NED), one of the red fluorescent proteins explored in `LaMBO` {cite:p}`stanton2022accelerating`. Before optimization, we need to download the file and "repair" it (see [single mutations using foldx](../../understanding_foldx/01-single-mutation-using-foldx/index.ipynb)).

We assume that the repaired file is already here.

In [3]:
!ls

3ned_Repair.pdb                    optimizing_protein_stability.ipynb


In [4]:
from pathlib import Path

wildtype_pdb_path = Path("./3ned_Repair.pdb").resolve()
wildtype_pdb_path.exists()  # Should say True

True

## Defining the objective function

In this tutorial, we optimize the stability of `mRogue` using the `foldx_stability` problem factory. The first step is creating the problem:

:::{warning}

In general, it is a good idea to check how to create instances of individual problems in their documentation, since they might need extra inputs.

For example, [`foldx_stability` only needs one extra keyword argument: a single/list of `wildtype_pdb_path`](../../objective_repository/foldx_stability.md). `poli` will hopefully remind you what you forgot with its error messages.

:::

In [5]:
from poli import objective_factory

f, x0, y0 = objective_factory.create(
    name="foldx_stability",
    wildtype_pdb_path=wildtype_pdb_path
)

poli 🧪: creating an isolated black box function.
poli 🧪: starting the isolated objective process.


`objective_factory.create` returns five things:

1. a `problem_info` with a description of the problem, including useful attributes like `alphabet` or `max_sequence_length`.
2. a black-box function `f: AbstractBlackBox` from `poli`.
3. an initial design `x0: np.ndarray`, and
4. an initial evaluation `y0: np.ndarray`.
5. `run_info`, or the output of the observer's initialization (see more [in our chapter about making observers](../../the_basics/defining_an_observer.ipynb)).

These are all the ingredients required for an abstract solver to work. The next section shows how to use a baseline solver, which can be easily replaced by any other solver you implement (as long as it inherits from the `AbstractSolver` in `poli_baselines.core.abstract_solver`).

## Optimizing using a `RandomMutation` solver

In this tutorial we use the simplest baseline for discrete sequence optimization: a `RandomMutation` which takes the best performing sequence and randomly mutates it by selecting a position at random, and altering for another element of the alphabet.

:::{note}

There's nothing special about `RandomMutation` here. You could drop-in any solver you implement as long as it

1. Inherits from `AbstractSolver` in `poli_baselines.core.abstract_solver`, and it
2. implements the abstract method `next_candidate() -> np.ndarray`.

[Check this tutorial on creating solvers for more details](../../the_basics/defining_a_problem_solver.md).

:::

In [6]:
from poli_baselines.solvers.simple.random_mutation import RandomMutation
solver = RandomMutation(
    black_box=f,
    x0=x0,
    y0=y0,
)

/Users/sjt972/anaconda3/envs/poli-docs2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**And that's it!** You can optimize the objective function passed as `black_box` by just calling the `.solve(n_iters)` method: (be careful, this might take a while)

In [7]:
solver.solve(max_iter=3)

## Checking the results

After optimization, the results are stored inside `solver.history`, which is a dictionary with `"x"` and `"y"` keys. Let's check what the best optimization result was:

In [8]:
print(f"All y values: {solver.history['y']}")
print(f"best stability: {solver.get_best_performance()}")
print(f"Associated sequence: {''.join(solver.get_best_solution().flatten())}")

All y values: [array([[9.41639]]), array([[5.56267]]), array([[0.846902]]), array([[9.39441]])]
best stability: [9.41639]
Associated sequence: EEDNMAIIKEFMRFKTHMEGSVNGHEFEIEGEGEGRPYEGTQTAKLKVTKGGPLPFAWDILSPQFSKAYVKHPADIPDYLKLSFPEGFKWERVMNFEDGGVVTVTQDSSLQDGEFIYKVKLRGTNFPSDGPVMQKKTMGWEACSERMYPEDGALKGEMKMRLKLKDGGHYDAEVKTTYKAKKPVQLPGAYNTNTKLDITSHNEDYTIVEQYERNEGRHSTGGMDELYK
